# Historical document : page and line extraction

In this tutorial you will use the _dhSegment_ package, which is a tool for segmentation of historical documents

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from skimage.transform import resize
from skimage import io
import tensorflow as tf

Before running this notebook, you need to install the dhSegment package.

`pip install git+https://github.com/dhlab-epfl/dhSegment`

In [ ]:
import dh_segment
from dh_segment.inference import LoadedModel
from dh_segment.io import PAGE
from dh_segment.post_processing import thresholding, cleaning_binary, cleaning_probs

### Downloading pre-trained models

The models that you will use were already trained (on the READ-BAD dataset), so you'll have to donwload them

Download and unzip `model.zip` from https://github.com/dhlab-epfl/dhSegment/releases/tag/v0.2 for a page extraction mdoel.

Download and unzip `line_model.zip` from https://github.com/dhlab-epfl/FDH_Tutorial_ComputerVision_DeepLearning/releases for a texline extraction model.

You need to load each model in a separated session

In [ ]:
sess1 = tf.InteractiveSession()
with sess1.graph.as_default():
    model_page = LoadedModel("model/")
    
sess2 = tf.InteractiveSession(graph=tf.Graph()) # New Graph need to be initialized in the session 2
with sess2.graph.as_default():
    model_textline = LoadedModel("polylines/")

Read and show the image that will be processed

In [ ]:
image_filename = "images/0167.jpg"

In [ ]:
img = io.imread(image_filename)
plt.figure(figsize=(10,10))
plt.imshow(img)

## Page extraction

You'll feed the image to the network. 

__WARNING__ : This step may take a few minutes

In [ ]:
output_page = model_page.predict(image_filename)

The `predict` method returns a dictionary with keys : 
* `probs` : the probability maps
* `original_shape` : the shape of the original image
* `labels` : the labels (the binary prediction, i.e thresholded `probs`)

The probability maps that is of interest for us in in the channel 1 (`output_page['probs'][0,:,:,1]`)

In [ ]:
page_probs = output_page['probs'][0,:,:,1]
plt.figure(figsize=(10,10))
plt.imshow(page_probs)

### Page map probabilty post-processing

First we need to threshold the probabilities to obtain a binary image (`thresholding`) and then the binary image is cleaned in order to remove small elements (`cleaning_binary`).

We then need to obtain the coordinates of the page 

In [ ]:
page_mask = thresholding(page_probs, threshold=0.7)
page_mask = cleaning_binary(page_mask, kernel_size=7).astype(np.uint8)*255
plt.figure(figsize=(10,10))
plt.imshow(page_mask)

### Find and save the coordinates of the page

In [ ]:
from dh_segment.post_processing import boxes_detection

In [ ]:
page_coords = boxes_detection.find_boxes(resize(page_mask, img.shape[:2]).astype(np.uint8), n_max_boxes=1)

The `PAGE.Pag`e object is a class representing a historical document page with its `TextRegion`s, `TextLine`s, `Border`, etc.

In [ ]:
PAGE_info = PAGE.Page(image_width=img.shape[1], image_height=img.shape[0],
                      page_border=PAGE.Border(PAGE.Point.list_to_point(list(page_coords))))

In [ ]:
plot_img = img.copy()
PAGE_info.draw_page_border(plot_img, autoscale=True, fill=False, thickness=15)
plt.figure(figsize=(10,10))
plt.imshow(plot_img)

## Texline detection

That that we have extracted the page, we'd like to extract the text lines, so again ge call `predict` function

In [ ]:
output_textline = model_textline.predict(image_filename)

In [ ]:
textline_probs = output_textline['probs'][0,:,:,1]
plt.figure(figsize=(10,10))
plt.imshow(textline_probs)

### Line map probabilty post-processing

We will use the binary mask we extracted in the previous section to eliminate the lines belonging to the left page that were wrongly detected.

In [ ]:
from dh_segment.post_processing import hysteresis_thresholding

textline_probs2 = cleaning_probs(textline_probs, 2)
extracted_page_mask = np.zeros(textline_probs.shape, dtype=np.uint8)
PAGE_info.draw_page_border(extracted_page_mask, color=(255,))
textline_mask = hysteresis_thresholding(textline_probs2, low_threshold=0.3, high_threshold=0.6,
                                        candidates_mask=extracted_page_mask>0
                                       )

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(textline_mask)

### Find the lines, vectorize and save them

In [ ]:
from dh_segment.post_processing import line_vectorization

lines = line_vectorization.find_lines(resize(textline_mask, img.shape[:2]))

In the `Page` structure, `TextLines` are always contained in `TextRegion` object, so first we create a `TextRegion` object which `TextLines` are the ones we just extracted.

In [ ]:
text_region = PAGE.TextRegion()
text_region.text_lines = [PAGE.TextLine.from_array(line) for line in lines]
PAGE_info.text_regions.append(text_region)

## Show the final results

In [ ]:
plot_img = img.copy()
PAGE_info.draw_page_border(plot_img, autoscale=True, fill=False, thickness=15)
PAGE_info.draw_lines(plot_img, autoscale=True, fill=False, thickness=15, color=(0,255,0))
plt.figure(figsize=(15,15))
plt.imshow(plot_img)